In [1]:
import os

from googleapiclient.discovery import build

In [2]:
# Arguments that need to passed to the build function
# API Key needs to be created in Google Cloud Console.- https://console.cloud.google.com/apis/api/youtube.googleapis.com/metrics
DEVELOPER_KEY = ''
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
   
# creating Youtube Resource Object
youtube_object = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                                        developerKey = DEVELOPER_KEY)

In [3]:
# Enter search keyword
keyword = "Naruto"

In [7]:
# Search 
search_keyword = youtube_object.search().list(q = keyword, part = "id, snippet",
                                               maxResults = 100).execute()
results = search_keyword.get("items", [])

### Extract information about videos, playlists, channels

In [8]:
videos = []
playlists = []
channels = []

for result in results:
    # videos
    if result['id']['kind'] == "youtube#video":
        videos.append({'title': result["snippet"]["title"],
                       'id': result["id"]["videoId"],
                       'decription': result['snippet']['description'],
                       'url': result['snippet']['thumbnails']['default']['url']})

    # playlists
    elif result['id']['kind'] == "youtube#playlist":
        playlists.append({'title': result["snippet"]["title"],
                          'id': result["id"]["playlistId"],
                          'decription': result['snippet']['description'],
                          'url': result['snippet']['thumbnails']['default']['url']})

    # channels
    elif result['id']['kind'] == "youtube#channel":
        channels.append({'title': result["snippet"]["title"],
                         'id': result["id"]["channelId"],
                         'decription': result['snippet']['description'],
                         'url': result['snippet']['thumbnails']['default']['url']})

In [9]:
print(f"Videos: {len(videos)}\nPlaylists: {len(playlists)}\nChannels: {len(channels)}")

Videos: 50
Playlists: 0
Channels: 0


### Extract Comments and replies from videos

In [10]:
def get_comment_threads(youtube, video_id, nextPageToken):
    results = youtube.commentThreads().list(
        part="snippet,replies",
        maxResults=100,
        videoId=video_id,
        textFormat="plainText",
        pageToken = nextPageToken
    ).execute()
    return results

In [11]:
def load_comments(video_response):
    
    comms = []
    for item in video_response['items']:
            
        # Extracting comments
        comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
        user_name = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
        user_url = item['snippet']['topLevelComment']['snippet']['authorChannelUrl']
        user_id = item['snippet']['topLevelComment']['snippet']['authorChannelId']
        comms.append({'text': comment, 'user': user_name, 'user_id': user_id, 'user_url': user_url})

        # counting number of reply of comment
        replycount = item['snippet']['totalReplyCount']

        if replycount > 0:
            if 'replies' not in item:
                continue
            for reply in item['replies']['comments']:
                comment = reply['snippet']['textDisplay']
                user_name = reply['snippet']['authorDisplayName']
                user_url = reply['snippet']['authorChannelUrl']
                user_id = reply['snippet']['authorChannelId']
                comms.append({'text': comment, 'user': user_name, 'user_id': user_id, 'user_url': user_url})
    return comms

In [12]:
comments = []

# Get comments from only top 10 search results
for video in videos[0:10]:
    
    video_id = video['id']
    video_response=get_comment_threads(youtube_object, video_id, '')
    next_page_token = video_response.get("nextPageToken", None)
    comments += load_comments(video_response)

    while next_page_token:
        video_response=get_comment_threads(youtube_object, video_id, next_page_token)
        next_page_token = video_response.get("nextPageToken", None)
        comments += load_comments(video_response)

In [13]:
len(comments)

5836

In [14]:
for comment in comments[:10]:
    print(comment['text'])

Ah yes, hinata family
Hi
I feel so angry
Bro... Hinata slap Naruto to cheer him
i just noticed that after pain destroyed the leaf and after Naruto beat him why didn't Nagato just use the thing that revives people (forgot the name) instead of rinne rebirth same with after Sasuke got rinnegan why didn't he just revive every one or most people that died in the war with the same thing that revives
plot
Watch my luffy vs naruto fan animation at https://www.youtube.com/watch?v=JVi4SIppBJw
Watch my luffy vs naruto fan animation at https://www.youtube.com/watch?v=JVi4SIppBJw
17:50 having Naruto done this just 1 minute earlier, Neji and a bunch of others (probably everyone) would be still standing😅
https://youtube.com/shorts/M1-uw9M5jjU?feature=share

Ghost of uchiha
